In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import necessary modules for Dash components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64

# Import CRUD module
from animal_shelter import AnimalShelter

# Define MongoDB connection details
username = "root"
password = "dUPEEMybzX"
host = "nv-desktop-services.apporto.com"  
port = "34745"  
database = "AAC"  
collection = "animals"

# Initialize MongoDB connection
db = AnimalShelter(username, password, host, port, database, collection)

# Read all records from MongoDB
df = pd.DataFrame.from_records(db.read({}))

# Drop MongoDB '_id' field to prevent errors in Dash
if "_id" in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Create Dash application
app = JupyterDash(__name__)

# Encode the image in Base64
image_filename = "grazioso_logo.png" 
with open(image_filename, "rb") as img_file:
    encoded_image = base64.b64encode(img_file.read()).decode()

# Dashboard Layout
app.layout = html.Div([
    # Branding & Logo
    html.Center(html.H1('CS-340 Animal Rescue Dashboard')),
    html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '200px'}),
    html.A("Grazioso Salvare", href="https://www.snhu.edu", target="_blank"),
    html.P("Created by Jeffrey Flores"), 

    html.Hr(),

    # Interactive Filter Options
    html.Label("Select Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'all'}
        ],
        value='all',
        labelStyle={'display': 'block'}
    ),

    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        sort_action="native",
        page_size=10,
        row_selectable="single",
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
    ),

    html.Br(),
    html.Hr(),

    # Charts & Geolocation
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# MongoDB Queries for Filtering
rescue_filters = {
    "water": {
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "mountain": {
        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "disaster": {
        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    }
}

# Callback for Filtering Data Table
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    query = rescue_filters.get(filter_type, {}) if filter_type != "all" else {}
    df_filtered = pd.DataFrame.from_records(db.read(query))
    
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)

    return df_filtered.to_dict('records')

# Callback for Updating Pie Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return "No data available"

    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Rescue Dog Breed Distribution')
        )
    ]

# Callback for Geolocation Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return "No data available"

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row,13], dff.iloc[row,14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

# Run Dashboard
app.run_server(debug=True)

